<a href="https://colab.research.google.com/github/jigyasa-grover/Hyper-Personalized-Ad-Campaigns-using-Generative-AI-Quick-Demo-using-Gemini-Pro-Vision/blob/main/Hyper_Personalized_Ad_Campaigns_using_Generative_AI_Gemini_Pro_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperpersonalized Ad Campaigns using Generative AI: Gemini Pro Vision

## Setup

In [1]:
!pip install -q google-generativeai==0.3.1
!pip install -q gradio

In [2]:
import google.generativeai as genai
from google.colab import userdata

from pathlib import Path
import gradio as gr

print(genai.__version__)
genai.configure(api_key = userdata.get('GEMINI_KEY'))

0.3.1


## Generative AI Model Configuration

In [3]:
GENERATION_CONFIG = {
  "temperature": 0.4,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

SAFETY_SETTINGS = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

## Generative AI Model Instance

In [4]:
GENERATIVE_AI_MODEL = genai.GenerativeModel(
    model_name = "gemini-pro-vision",
    generation_config = GENERATION_CONFIG,
    safety_settings = SAFETY_SETTINGS
)

## Inut Prompt

In [5]:
NUM_USER_PERSONAS = 4

BASE_INPUT_PROMPT = """
  We want to run hyper-personalized online ad campaigns for the product in the picture above.
  Create ad campaign strategies that target the following four user personas.
  Each strategy should include suggestions for compelling ad copy that would appeal strongly to the specific persona.
  Tailor the ad campaigns to address the specific needs, interests, and pain points of each persona.
  Consider using dynamic ad elements, product details and real-time data for even more granular personalization.
  Please include a quirky personalized hashtag in the ad campaign text. The ad campaign text should not be more than 150 words.
"""
# + user_persona_prompt (parsed using the parse_persona_string(..) helper fn)

## Helper Functions

In [6]:
def process_input_image(file_location):
    if not (img := Path(file_location)).exists():
        raise FileNotFoundError(f"Could not find image: {img}")
    image = [{
          "mime_type": "image/jpeg",
          "data": Path(file_location).read_bytes()
      }]
    return image

In [7]:
def render_input_image(input_files):
    input_files_paths = [file.name for file in input_files]
    return input_files_paths[0]

In [8]:
def parse_persona_string(user_persona_instance, user_persona_string):
    user_persona = user_persona_string.split(",")
    parsed_user_persona = "\n\t".join([
        f"{key.capitalize()}: {value}" for key, value in zip(
              ["Age", "Gender", "Location", "Occupation"], user_persona
            )
        ])
    return f"\nUser Persona {user_persona_instance}:\n\t{parsed_user_persona}"

In [9]:
def generate_genAI_model_response(input_prompt, user_personas, input_image_location):
    image_prompt = process_input_image(input_image_location)
    user_personas_prompt = ""
    for i in range(NUM_USER_PERSONAS):
      user_personas_prompt += parse_persona_string(i+1, user_personas[i])
    prompt_parts = [input_prompt + user_personas_prompt, image_prompt[0]]
    response = GENERATIVE_AI_MODEL.generate_content(prompt_parts)
    return response.text

In [10]:
def output_image_and_genAI_model_response(files, *user_personas):
    file_paths = [file.name for file in files]
    if file_paths:
        response = generate_genAI_model_response(BASE_INPUT_PROMPT, user_personas, file_paths[0])
    return file_paths[0], response

In [ ]:
with gr.Blocks() as demo:
    header = gr.Markdown(
        """
          # Hyperpersonalized Ad Campaigns using Generative AI: Gemini Pro Vision
        """)
    user_personas_textboxes = [
        gr.Textbox(label="User Persona #" + str(i+1), placeholder = "age, gender, location, profession")
        for i in range(NUM_USER_PERSONAS)
    ]
    image_upload_button = gr.UploadButton("Click to upload an image of the product to be advertised.", file_types=["image"], file_count="multiple")

    input = [image_upload_button]
    for i in range(NUM_USER_PERSONAS):
      input.append(user_personas_textboxes[i])

    image_output = gr.Image()

    generate_button = gr.Button(value="Generate!")

    genAI_model_response = gr.Markdown()
    output = [image_output, genAI_model_response]

    image_upload_button.upload(render_input_image, image_upload_button, image_output)
    generate_button.click(output_image_and_genAI_model_response, input, output)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3bb74fac25923349bd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
